In [35]:
import sys
sys.path.append('/Users/venkatmanavarthi/Desktop/torch_bot')

In [36]:
import nltk
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from utils.utils import tokenize, stem, bag_of_words

# Data Pipeline

In [37]:
# file paths
INTENTS_FILE = "../shared/intents.json" # intents

In [38]:
with open(INTENTS_FILE, 'r') as f:
    intents = json.load(f)

# print(intents)

In [39]:
# data variables
tags = []
all_words = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    responses = intent['responses']
    patterns = intent["patterns"]
    for pattern in patterns:
        # tokenizing the string or sentense input 
        tok = tokenize(pattern)
        all_words.extend(tok)
        xy.append((tok, tag))
ignore_words = ['!', '~', '.', '?', ',']

# stemming
stemmed_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(stemmed_words))
tags = sorted(set(tags))

In [40]:
tags

['goodbye', 'greeting', 'noanswer', 'thanks']

In [41]:
X_train = []
y_train = []



for (token, tag) in xy:
    bag = bag_of_words(token, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)


X_train = np.array(X_train)
y_train = np.array(y_train)

# Dataset, Dataloader

In [42]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_train = y_train
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_train[index]

    def __len__(self):
        return self.n_samples


# Neural Network Training

In [43]:
from model import NeuralNet

In [44]:
### hyperparematers 
batch_size = 8
input_size = len(X_train[0])
output_size = len(tags)
hidden_size = 8
dataset = ChatDataset()
learning_rate = 0.001
num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True) #num_workers=2)
model = NeuralNet(input_size, hidden_size, output_size)

In [45]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "../shared/data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

Epoch [100/100], Loss: 0.6902
final loss: 0.6902
training complete. file saved to ../shared/data.pth
